In [1]:
import weka.core.jvm as jvm
from weka.core.converters import Loader
import pandas as pd
import numpy as np

In [2]:
jvm.start()

loader = Loader(classname="weka.core.converters.ArffLoader")
data = loader.load_file("input/L3_1.arff")
data.class_is_last()

data_list = [[inst.get_value(attr.index) for attr in data.attributes()] for inst in data]
df = pd.DataFrame(data_list, columns=[attr.name for attr in data.attributes()])
data_list = [[inst.get_value(attr.index) for attr in data.attributes()] for inst in data]
df = pd.DataFrame(data_list, columns=[attr.name for attr in data.attributes()])

# print(df.head())

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\core.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\mtj.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'C:\\Users\\micha\\anaconda3\\envs\\weka_env\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [3]:
def calculate_entropy(df, column):
    counts = df[column].value_counts()
    probabilities = counts / len(df)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy


def calculate_gain_ratio(df, attribute, class_col):
    total_entropy = calculate_entropy(df, class_col)
    values = df[attribute].unique()

    weighted_entropy = 0
    split_info = 0
    for value in values:
        subset = df[df[attribute] == value]
        prob = len(subset) / len(df)
        weighted_entropy += prob * calculate_entropy(subset, class_col)
        split_info -= prob * np.log2(prob) if prob > 0 else 0

    information_gain = total_entropy - weighted_entropy
    gain_ratio = information_gain / split_info if split_info != 0 else float('inf')

    return gain_ratio


attribute_gain_ratios = {col: calculate_gain_ratio(df, col, class_col='status pozyczki') for col in df.columns if col != 'status pozyczki'}
# print(attribute_gain_ratios)

with open('output/attributes.txt', 'w') as f:
    for attribute, gain_ratio in attribute_gain_ratios.items():
        f.write(f"{attribute}: {gain_ratio:.4f}\n")
jvm.stop()

https://pl.wikipedia.org/wiki/Entropia_(teoria_informacji)
log o podstawie 2 dla bitowych decyzji